In [3]:
import os
trees=["NOMINAL"]
with open("../../sys_trees.txt","r") as file:
    line=file.readline()
    while line:
        trees.append(line.rstrip('\n'))
        line=file.readline()

samples=["Signal_Sherpa","Signal_PoPy","Zmumu","VV"]

In [2]:

import ROOT as r
dict={}

# FILLING DICTIONARY WITH SYS-RELATED TREES 

### the dictionary will have the following structure "systematic_name":[fake1p,truth1p,fake3p,truth3p]

for sam in samples:
    for sys in trees[1:]:
        file=r.TFile.Open(sam+sys+".root")
        prong1=file.Get("tau_matched_1p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")
        prong3=file.Get("tau_matched_3p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")
        dict[sam+sys]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        file.Close()



Welcome to JupyROOT 6.22/00


ReferenceError: attempt to access a null-pointer

In [8]:
#NOW ADDING THE WEIGHT-BASED SYSTEMATICS        
histograms=[]
for sys in trees[:1]:
    file=r.TFile.Open("Signal_Sherpa"+sys+".root")
    histos=file.GetListOfKeys()
    for i in histos:
        name=i.GetName()
        if ("cuts" in name) and ("tpt" in name) and ("1p" in name):
            histograms.append(i.GetName()[len("tau_matched_1p_cuts_tpt_"):])

for sam in samples:
    for sys in trees[:1]:
        file=r.TFile.Open(sam+sys+".root")
        for h in histograms:
            prong1=file.Get("tau_matched_1p_cuts_tpt_"+h)
            prong3=file.Get("tau_matched_3p_cuts_tpt_"+h)
            dict[sam+sys+h]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        prong1=file.Get("tau_matched_1p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")
        prong3=file.Get("tau_matched_3p_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_tpt")
        dict[sam+sys+"nominal"]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        file.Close()

In [9]:
# REARRANGING THE DICTIONARY AS FAKES AND TRUTH EVENTS
for i in dict:
    if "Signal" not in i:
        dict[i]=[dict[i][0]+dict[i][1],0,dict[i][2]+dict[i][3],0]

In [10]:
#The structure of the dictionary is... "name_of_sys":[SherpaFake,SherpaTruth,PoPyFake,PoPyTruth,Zmumu,VV]
final_dict_1p={}
for i in histograms+trees[1:]:
    final_dict_1p[i]=[0,0,0,0,0,0]
    for j in dict:
        if (i in j) and ("Sherpa" in j):
            final_dict_1p[i][0]=dict[j][0]
            final_dict_1p[i][1]=dict[j][1]
        if (i in j) and ("PoPy" in j):
            final_dict_1p[i][2]=dict[j][0]
            final_dict_1p[i][3]=dict[j][1]
        if (i in j) and ("Zmumu" in j):
            final_dict_1p[i][4]=dict[j][0]
        if (i in j) and ("VV" in j):
            final_dict_1p[i][5]=dict[j][0]
            
final_dict_3p={}
for i in histograms+trees[1:]:
    final_dict_3p[i]=[0,0,0,0,0,0]
    for j in dict:
        if (i in j) and ("Sherpa" in j):
            final_dict_3p[i][0]=dict[j][2]
            final_dict_3p[i][1]=dict[j][3]
        if (i in j) and ("PoPy" in j):
            final_dict_3p[i][2]=dict[j][2]
            final_dict_3p[i][3]=dict[j][3]
        if (i in j) and ("Zmumu" in j):
            final_dict_3p[i][4]=dict[j][2]
        if (i in j) and ("VV" in j):
            final_dict_3p[i][5]=dict[j][2]

#TAKING DOWN AND UP SYSTEMATICS TO THE BOTTOM AND THE TOP OF THE DICTIONARY RESPECTIVELY
down_1p={}
down_3p={}

a1p_keys=[i for i in final_dict_1p]
a3p_keys=[i for i in final_dict_3p]

for i in a1p_keys:
    if "1down" in i:
        down_1p.update({i:final_dict_1p[i]})
        final_dict_1p.pop(i)
for i in a3p_keys:
    if "1down" in i:
        down_3p.update({i:final_dict_3p[i]})
        final_dict_3p.pop(i)
final_dict_1p.update(down_1p)
final_dict_3p.update(down_3p)

In [11]:
# EXPORTING THE RESULTS TO A CSV FILE
import pandas as pd
df1=pd.DataFrame.from_dict(final_dict_1p, orient='index',
                       columns=["SherpaFake","SherpaTruth","PoPyFake","PoPyTruth","Zmumu","VV"])
df3=pd.DataFrame.from_dict(final_dict_3p, orient='index',

                       columns=["SherpaFake","SherpaTruth","PoPyFake","PoPyTruth","Zmumu","VV"])

df1.to_csv('Weight-Based-Uncertainties-1prong.csv')
df3.to_csv('Weight-Based-Uncertainties-3prong.csv')
